## stitch HV data ( image & rough label) in MyoSegmenTUM & update csv
- 發現 MyoSegmenTUM 下的 `HV 開頭` 的 data 是分段的，所以嘗試將兩段黏起來


### Configuration **將 csv column name 中具有空格的地方改成 "_" **

In [ ]:
import os
import shutil
import pandas as pd
from collections import defaultdict
import pandas as pd
import re
import numpy as np
import nibabel as nib
from collections import defaultdict
from scipy.ndimage import zoom
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import glob
import random
import numpy.ma as ma

# --- 全域設定 (Global Configurations) ---
# 1. 專案根目錄 (Project Root)
PROJECT_ROOT = "/home/n26141826/114-1_TAICA_cv_Final_Project"

# 2. 原始資料設定 (Raw Data Input)
RAW_DATA_FOLDER = os.path.join(PROJECT_ROOT, "data", "data")
RAW_CSV_FILE = os.path.join(RAW_DATA_FOLDER, "metadata_3D.csv")

# 3. 拼接處理設定 (Stitching Output)
# 拼接後的 3D 檔案與新 CSV 要存哪裡
FIXED_DATA_ROOT = os.path.join(PROJECT_ROOT, "data2", "data_fixed")
OUTPUT_CSV_FILE = os.path.join(FIXED_DATA_ROOT, "metadata_3D_stitched.csv")

# 4. 2D 切片輸出設定 (2D Slices Output)
# 這是最終 PyTorch Dataset 要讀取的 .npy 檔案位置
OUTPUT_SLICE_DIR = os.path.join(PROJECT_ROOT, "data2", "npy_Embedding")

# 5. csv 欄位名稱設定 (CSV Column Names)
DATASET = "Dataset"
PHENOTYPE = "Phenotype"
MRI_SAMPLRE = "MRI_sample" 
MRI_SEQUENCE = "MRI_Sequence"                 # 我把空格改成底線了
IMAGE_3D_FILE = "image_3D_file"               # 我把空格改成底線了
ROUGH_LABEL_3D_FILE = "rough_label_3D_file"
DETAILED_LABEL_3D_FILE = "detailed_label_3D_file"
# 6. 資料篩選條件 (Data Filter)
TARGET_DATASET = 'MyoSegmenTUM'
TARGET_PHENOTYPE = 'Control'

# 7. MRI 序列映射表 (Modality Mapping)
# String -> Int ID
TYPE_MAP = {
    'Water': 0,
    'Fat': 1,
    'FATFRACTION': 1, # 通常將 Fat Fraction 視為 Fat 類別，或依你需求改為獨立 ID
    'T1': 2,
    'T2': 3,
    'STIR': 4
}
# position-embedding 用的類別數量 
# -> 0 : 屁股
# -> 1 : 膝蓋

POSITION_ROUGH_EMBEDDING_COLORS = [
    '#000000', # 0: BG
    '#e6194b', # 1: SA
    '#006400', # 2: RF (Green)
    '#228B22', # 3: VL (Green)
    '#32CD32', # 4: VI (Green)
    '#7CFC00'  # 5: VM (Green)
]
POSITION_DETAILED_EMBEDDING_COLORS = [
    '#000000', # 0: BG
    '#e6194b', # 1: SA
    '#006400', # 2: RF (Green)
    '#228B22', # 3: VL (Green)
    '#32CD32', # 4: VI (Green)
    '#7CFC00', # 5: VM (Green)
    '#911eb4', # 6: AM (Purple)
    '#46f0f0', # 7: GR (Cyan)
    '#00008B', # 8: BFL (Blue)
    '#0000CD', # 9: ST (Blue)
    '#4169E1', # 10: SM (Blue)
    '#87CEEB'  # 11: BFS (Blue)
]
ROUGH_MUSCLE_NAMES = {
    0: 'Background', 1: 'Sartorius', 2: 'Rectus Femoris', 3: 'Vastus Lateralis',
    4: 'Vastus Intermedius', 5: 'Vastus Medialis'
}
DETAIL_MUSCLE_NAMES = {
    0: 'Background', 1: 'Sartorius', 2: 'Rectus Femoris', 3: 'Vastus Lateralis',
    4: 'Vastus Intermedius', 5: 'Vastus Medialis', 6: 'Adductor Magnus',
    7: 'Gracilis', 8: 'Biceps Femoris LH', 9: 'Semitendinosus',
    10: 'Semimembranosus', 11: 'Biceps Femoris SH'
}
# --- 自動建立資料夾 (Auto-create Dirs) ---
for d in [FIXED_DATA_ROOT, OUTPUT_SLICE_DIR,OUTPUT_SLICE_DIR+'/train',OUTPUT_SLICE_DIR+'/test']:
    os.makedirs(d, exist_ok=True)

print("✅ Configuration Loaded Successfully!")
print(f"- Raw CSV: {RAW_CSV_FILE}")
print(f"- Output Stitched CSV: {OUTPUT_CSV_FILE}")
print(f"- Final 2D Output Dir: {OUTPUT_SLICE_DIR}")

### 1. copy original dataset (2D+3D)

In [ ]:
if os.path.exists(FIXED_DATA_ROOT) and os.path.isdir(FIXED_DATA_ROOT):
    shutil.rmtree(FIXED_DATA_ROOT)
    
shutil.copytree(RAW_DATA_FOLDER, FIXED_DATA_ROOT)
print(f"Copied dataset folder to {FIXED_DATA_ROOT}")


### 2. delete whole MyoSegmenTUM in copied 3D file (image + rough label)

In [ ]:

# read csv from copied folder and write new csv into fixed folder
df = pd.read_csv(RAW_CSV_FILE, sep=',')
# print(df.head(2))
for index, row in df.iterrows():
    dataset = row[DATASET]
    # process path
    if dataset == TARGET_DATASET:
        img_path = row[IMAGE_3D_FILE]
        rough_path = row[ROUGH_LABEL_3D_FILE]
        img_full_path = os.path.join(FIXED_DATA_ROOT, img_path.replace('\\', '/'))
        rough_full_path =  os.path.join(FIXED_DATA_ROOT, rough_path.replace('\\', '/'))
        # delete image and rough label .nii.gz files
        if os.path.exists(img_full_path):
            os.remove(img_full_path)
            print(f"Deleted image file: {img_full_path}")
        if os.path.exists(rough_full_path):
            os.remove(rough_full_path)
            print(f"Deleted rough label file: {rough_full_path}")


### 3. stitch HV data in MyoSegmenTUM
### 4. add them in to 3D dataset 
### 5. fix `metadata_3D.csv` in copied folder 

In [ ]:
class CsvThighStitcher:
    def __init__(self, csv_path):
        self.csv_path = csv_path
        # Regex: (排序號碼)_(群組名稱).nii
        self.pattern = re.compile(r"^(\d+)_(.+)\.nii(\.gz)?$")

    def parse_csv_and_group(self, data_folder):
        """
        讀取 CSV 並將 Image 與 Label 綁定在一起分組
        Return:
            groups: dict
            Key = 'HV011_FATFRACTION'
            Value = list of tuples: (sort_num, img_path, rough_path, raw_row_series)
        """
        print(f"Reading CSV: {self.csv_path}")
        df = pd.read_csv(self.csv_path, sep=',') 
        
        # Filter
        target_df = df[(df[DATASET] == TARGET_DATASET) & (df[PHENOTYPE] == TARGET_PHENOTYPE)]
        print(f"Found {len(target_df)} rows for 'MyoSegmenTUM' & 'Control'.")

        groups = defaultdict(list)
        
        for idx, row in target_df.iterrows():
            img_path = row[IMAGE_3D_FILE]
            rough_path = row[ROUGH_LABEL_3D_FILE]
            
            # 處理路徑分隔符號
            imgname = img_path.replace('\\', '/').split('/')[-1]
            # roughname 其實不需要 parse regex，因為通常跟 imgname 是對應的，
            # 我們假設 imgname 的 sort_num 就是這組資料的順序
            
            img_path = img_path.replace('\\', '/')
            rough_path = rough_path.replace('\\', '/')
            
            # 使用 Regex 解析檔名
            match = self.pattern.match(imgname)
            
            if match:
                sort_num = int(match.group(1)) # 轉成 int 以便排序
                group_key = match.group(2)     # 例如 "HV011_FATFRACTION"
                
                # 關鍵修改：將 Img, Rough, 和 原始Row 全部打包
                groups[group_key].append((sort_num, img_path, rough_path, row))
                
                # print(f"Parsed: {imgname} -> Key: {group_key}, Sort: {sort_num}")
            else:
                print(f"[Warning] Filename pattern mismatch: {imgname}")

        return groups

    def process_groups(self, groups):
        """
        執行拼接 (同時拼接 Image 和 Rough Label)
        """            
        valid_pairs = 0
        orphans = []
        ambiguous = []
        
        # 用來收集拼接後的資料，傳給 update_csv 使用
        # 結構: list of new rows (Series)
        stitched_rows_data = [] 

        print(f"\n--- Starting Stitching Process ---")

        for key, items in groups.items():
            count = len(items)
            
            if count == 2:
                # === 完美配對 (Case A) ===
                # items 是 list of (sort_num, img_path, rough_path, row)
                # 根據 sort_num 排序: 小(Upper) -> 大(Lower)
                items.sort(key=lambda x: x[0]) 
                
                upper_item = items[0] 
                lower_item = items[1]
                
                # 1. 拼接 Image
                success_img, output_img_name = self.stitch_pair(
                    upper_item[1], lower_item[1], # img paths
                    upper_item[0], key,           # sort_num, key
                    is_label=False
                )
                
                # 2. 拼接 Label (Order=0 Nearest Neighbor)
                success_rough, output_rough_name = self.stitch_pair(
                    upper_item[2], lower_item[2], # rough paths
                    upper_item[0], key,           # sort_num, key
                    is_label=True
                )

                if success_img and success_rough:
                    valid_pairs += 1
                    
                    # === 準備更新 CSV 的資料 ===
                    # 複製 Upper 的原始資料當作基底
                    new_row = upper_item[3].copy()
                    
                    # 產生新的檔名 (需與 stitch_pair 中的存檔邏輯一致)
                    # 邏輯: {Upper_Sort_Num}_full_{Key}.nii.gz
                    new_filename = f"{upper_item[0]}_full_{key}.nii.gz"
                    
                    # 更新路徑 (存相對路徑或絕對路徑皆可，這裡示範存 output 資料夾下的路徑)
                    new_row[IMAGE_3D_FILE] = output_img_name.replace('/', '\\')
                    new_row[ROUGH_LABEL_3D_FILE] = output_rough_name.replace('/', '\\')
                    
                    stitched_rows_data.append(new_row)
                    
            elif count < 2:
                orphans.append(key)
            else:
                ambiguous.append((key, count))

        # --- Report ---
        print(f"\nProcessed {valid_pairs} pairs successfully.")
        if orphans: 
            print(f"[Warning] {len(orphans)} orphans found. -- non added to output folder.")
            for orphan in orphans:
                print(f"  - Orphan group: {orphan}")
        if ambiguous: 
            print(f"[Warning] {len(ambiguous)} ambiguous groups found. -- none added to output folder.")
            for ambi in ambiguous:
                print(f"  - Ambiguous group: {ambi[0]} with {ambi[1]} items")
        
        return stitched_rows_data

    def stitch_pair(self, subpath_u, subpath_l, sort_num, group_key, is_label=False):
        """
        通用拼接函式 (可處理 Image 或 Label)
        """
        try:
            path_l = os.path.join(RAW_DATA_FOLDER, subpath_l)
            path_u = os.path.join(RAW_DATA_FOLDER, subpath_u)
            print(f"subpath_u: {subpath_u}")
            if not os.path.exists(path_u) or not os.path.exists(path_l):
                print(f"[Error] File missing for {group_key} - {path_u} or {path_l} not found.")
                return False

            img_u = nib.load(path_u)
            img_l = nib.load(path_l)
            data_u = img_u.get_fdata()
            data_l = img_l.get_fdata()
            
            # Resize logic
            if data_u.shape[:2] != data_l.shape[:2]:
                target_h = min(data_u.shape[0], data_l.shape[0])
                target_w = min(data_u.shape[1], data_l.shape[1])
                # Label 用 order=0 (Nearest), Image 用 order=1 (Linear)
                order = 0 if is_label else 1
                data_u = resize_image(data_u, (target_h, target_w, data_u.shape[2]), order)
                data_l = resize_image(data_l, (target_h, target_w, data_l.shape[2]), order)
            
            # 拼接
            data_l = data_l[:, :, :-1]
            data_u = data_u[:, :, 1:]
            stitched_data = np.concatenate([data_l, data_u], axis=2)
            
            # 存檔
            new_img = nib.Nifti1Image(stitched_data, img_u.affine, img_u.header)
            save_name = f"{sort_num}_{group_key}.nii.gz"
            output_name = os.path.join(os.path.dirname(subpath_l), save_name)
            save_path = os.path.join(FIXED_DATA_ROOT, output_name)
            nib.save(new_img, save_path)
            print(f"[Info] Stitched saved: {save_path}")
            
            return True, output_name
        except Exception as e:
            print(f"[Error] Stitching failed for {group_key}: {e}")
            return False, None

def resize_image(image, target_shape, order=1):
    current_shape = image.shape
    zoom_factors = [
        target_shape[0] / current_shape[0],
        target_shape[1] / current_shape[1],
        target_shape[2] / current_shape[2]
    ]
    return zoom(image, zoom_factors, order=order)

def update_csv_direct(original_csv_path, output_csv_path, new_rows_list):
    """
    直接接收處理好的 new_rows_list 並寫入新的 CSV
    """
    print(f"\n--- Generating Updated CSV: {output_csv_path} ---")
    
    # 1. 讀取原始 CSV (為了保留那些 "不需要拼接" 的資料，例如其他 Dataset 的資料)
    df = pd.read_csv(original_csv_path)
    
    # 2. 移除舊的 "MyoSegmenTUM + Control" 資料
    # 我們要用新生成的 stitched rows 來取代它們
    # 條件: Dataset不是MyoSegmenTUM 或者 Phenotype不是Control 的資料要保留
    df_kept = df[~(df[DATASET] == TARGET_DATASET)]
    
    # 3. 建立新資料的 DataFrame
    if new_rows_list:
        df_new = pd.DataFrame(new_rows_list)
        
        # 4. 合併 (保留的舊資料 + 拼接後的新資料)
        df_final = pd.concat([df_kept, df_new], ignore_index=True)
        
        # 5. 存檔
        df_final.to_csv(output_csv_path, index=False)
        print(f"Success! Updated CSV saved with {len(df_final)} rows.")
    else:
        print("No new stitched rows to update. CSV remains unchanged.")

In [ ]:
# 設定路徑
if not os.path.exists(RAW_CSV_FILE):
    print("CSV file not found.")
    exit()

stitcher = CsvThighStitcher(RAW_CSV_FILE)

# 1. Parse & Group (只跑一次，同時包含 Img 和 Rough 的資訊)
grouped_data = stitcher.parse_csv_and_group(RAW_DATA_FOLDER)

# 2. Process & Stitch (回傳可以用來寫入 CSV 的 rows)
stitched_rows = stitcher.process_groups(grouped_data)

# 3. Update CSV (直接拿上面的結果來存，不用再 parse 一次)
update_csv_direct(RAW_CSV_FILE, OUTPUT_CSV_FILE, stitched_rows)

### 6. convert 3D to 2D & add position, measure_type embedding 

In [ ]:
def get_type_id(sequence_name):
    """將文字類別轉為整數 ID"""
    # 移除可能的空白並轉 Title case (e.g., " water " -> "Water")
    clean_name = str(sequence_name).strip()
    # 模糊比對或直接查表
    for key, val in TYPE_MAP.items():
        if key.upper() in clean_name.upper():
            return val
    return -1 # 未知類別

# ==========================================
# 2. 執行切片與 Embedding 注入
# ==========================================
print(f"Reading CSV: {OUTPUT_CSV_FILE}")
df = pd.read_csv(OUTPUT_CSV_FILE)
print(f"Total subjects to process: {len(df)}")
fail_files = []
success_count = 0
fail_count = 0

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Slicing 3D Volumes"):
    # 1. 取得路徑 (修正 Windows/Linux 分隔符號)
    sub_img_path = str(row[IMAGE_3D_FILE]).replace('\\', '/')
    sub_rough_lbl_path = str(row[ROUGH_LABEL_3D_FILE]).replace('\\', '/')
    sub_detail_lbl_path = str(row[DETAILED_LABEL_3D_FILE]).replace('\\', '/')
    img_path = os.path.join(FIXED_DATA_ROOT, sub_img_path) 
    rough_lbl_path = os.path.join(FIXED_DATA_ROOT, sub_rough_lbl_path)
    
    has_detail = False
    detail_path = None
    detail_lbl_path = os.path.join(FIXED_DATA_ROOT, sub_detail_lbl_path)
    print(f"detail_lbl_path: {detail_lbl_path}")
    if os.path.exists(detail_lbl_path):
        has_detail = True
    
    # 取得 MRI 序列類別
    seq_name = row.get(MRI_SEQUENCE) # 假如 CSV 有這欄
    type_idx = get_type_id(seq_name)
    MRI_sample = row.get(MRI_SAMPLRE, "")
    
    print(f"MRI_sample: {MRI_sample}")
    
    try:
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"File not found: {img_path}")

        # 2. 讀取 NIfTI
        img_obj = nib.load(img_path)
        rough_lbl_obj = nib.load(rough_lbl_path)
        detail_lbl_obj = nib.load(detail_lbl_path) if has_detail else None
        
        # 轉為 Numpy Array (H, W, D)
        img_vol = img_obj.get_fdata()
        rough_lbl_vol = rough_lbl_obj.get_fdata()
        detail_lbl_vol = detail_lbl_obj.get_fdata() if has_detail else np.zeros_like(rough_lbl_vol)
        
        vol_max = np.max(img_vol)
        
        # 3. 獲取維度資訊
        h, w, d = img_vol.shape
        filename = os.path.basename(img_path)
        
        # 4. 開始切片 (Slice along Z-axis)
        for z in range(d):
            # --- A. 提取資料 ---
            slice_img = img_vol[:, :, z]
            slice_rough_lbl = rough_lbl_vol[:, :, z]
            slice_detail_lbl = detail_lbl_vol[:, :, z]
            
            # 刪掉屁股附近全黑的切片
            if np.max(slice_rough_lbl) == 0:
                print(f"[Info] Skipping slice {z} of {filename} due to empty rough label.")
                # 跳過全黑標註或有 NaN 的切片
                continue
            
            slice_img = np.rot90(slice_img, k=-1)
            slice_rough_lbl = np.rot90(slice_rough_lbl, k=-1)
            slice_detail_lbl = np.rot90(slice_detail_lbl, k=-1)
            
            new_h, new_w = slice_img.shape
            
            # --- B. 數值正規化 (Normalization) ---
            # 雖然不改大小，但數值建議先縮放到 0-1，避免存成 float32 時數值過大
            if vol_max > 0:
                slice_img = slice_img / vol_max
            
            # --- C. 計算 Embedding ---
            # Position Embedding: 相對位置 (0.0 ~ 1.0)
            z_pos = 1.0 - (z / (d - 1)) if d > 1 else 0.0
            # z_pos = 1-z_pos  # 反轉位置，0.0 = 屁股, 1.0 = 膝蓋
            
            
            # 每位病患的每個 slice 為一個 npy 檔案，包含不同量測的 image 與 一個 label
            save_data = {
                "image_Water": slice_img.astype(np.float32),       # (H, W) 原尺寸
                "image_Fat": slice_img.astype(np.float32),         # (H, W) 原尺寸
                "image_T1": slice_img.astype(np.float32),          # (H, W) 原尺寸
                "image_T2": slice_img.astype(np.float32),          # (H, W) 原尺寸
                "image_STIR": slice_img.astype(np.float32),        # (H, W) 原尺寸
                "image_FATFRACTION": slice_img.astype(np.float32), # (H, W) 原尺寸
                "rough_label": slice_rough_lbl.astype(np.uint8),    
                "detail_label": slice_detail_lbl.astype(np.uint8), 
                "has_detail": np.bool_(has_detail),                # 是否有詳細標註
                "z_pos": np.float32(z_pos),                        # 
                "this_slice": int(z+1),                            # 純量
                "total_slices": int(d),                            # 純量
            }
            
            # --- E. 存檔 (.npy) ---
            # 檔名範例: 533_1_HV014_FAT.npy
            # file_name = os.path.splitext(filename)[0].split('_')
            slice_name = os.path.splitext(filename)[0].split('.')[0].split('_')            
            save_name = f"{slice_name[0]}_{type_idx}_{'_'.join(slice_name[1:])}_slice{z:03d}.npy"
            save_path = os.path.join(OUTPUT_SLICE_DIR, sub_img_path.split('/')[-3], str(save_name))
            print(f"[Info] Saved slice: {save_path}")
            
            np.save(save_path, save_data)
            
        success_count += 1

    except Exception as e:
        print(f"[Error] Failed {img_path}: {e}")
        fail_files.append(img_path)
        fail_count += 1

print(f"\nProcessing Done!")
print(f"Success Volumes: {success_count}")
print(f"Failed Volumes : {fail_count}")
print(f"Failed Files  : {fail_files}")
print(f"Saved to: {OUTPUT_SLICE_DIR}")

In [ ]:
# 設定你的 .npy 資料夾 (請確認路徑是否正確，例如是否在 train/image 底下)
npy_dir = os.path.join(OUTPUT_SLICE_DIR,'train')

cmap_rough = mcolors.ListedColormap(POSITION_ROUGH_EMBEDDING_COLORS)
cmap_detailed = mcolors.ListedColormap(POSITION_DETAILED_EMBEDDING_COLORS)


# 1. 搜尋檔案
files = glob.glob(os.path.join(npy_dir, "*.npy"))
print(f"Found {len(files)} .npy slices.")

if len(files) == 0:
    print("Error: No files found. Check your directory path.")
else:
    # 2. 隨機抽樣 3 張來檢查
    samples = random.sample(files, 10)

    for i, fpath in enumerate(samples):
        print(f"\n" + "="*50)
        print(f"Checking Sample {i+1}: {os.path.basename(fpath)}")
        
        # 讀取
        data = np.load(fpath, allow_pickle=True).item()
        
        # 提取資料 (注意 Key 名稱需對應存檔時的設定)
        img = data['image']
        lbl_rough = data.get('rough_label')   # 粗標註
        lbl_detail = data.get('detail_label') # 細標註
        has_detail = data.get('has_detail', False)
        z_pos = data.get('z_pos', -1)
        type_idx = data.get('type_idx', -1)
        MRI_sample = data.get('MRI_sample', 'Unknown')
        this_slice = data.get('this_slice', -1)
        total_slices = data.get('total_slices', -1)
        print(f"- z_pos: {z_pos:.4f}, type_idx: {type_idx}, has_detail: {has_detail}, MRI sample: {MRI_sample}, Slice: {this_slice}/{total_slices}")
        
        # --- 建立畫布 (1列 3行) ---
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # 共用的顯示設定
        title_fontsize = 12
        
        # === Plot 1: 原圖 (Raw Image) ===
        axes[0].imshow(img, cmap='gray')
        axes[0].set_title(f"Raw Image\nType: {type_idx}, Z: {z_pos:.2f}", fontsize=title_fontsize)
        axes[0].axis('off')
        
        # === Plot 2: 原圖 + Rough Label (Overlay) ===
        axes[1].imshow(img, cmap='gray') # 先畫底圖
        if lbl_rough is not None:
            # 將背景 (0) 遮罩起來，使其完全透明
            masked_rough = ma.masked_where(lbl_rough == 0, lbl_rough)
            # 疊加 Label (alpha=0.5 半透明, cmap='tab10' 顏色對比高)
            axes[1].imshow(masked_rough, cmap=cmap_rough, alpha=0.5, interpolation='nearest')
            axes[1].set_title(f"Overlay: Rough Label\n(5 Classes)", fontsize=title_fontsize)
        else:
            axes[1].text(0.5, 0.5, "No Rough Label", ha='center', color='red')
        axes[1].axis('off')
        
        # === Plot 3: 原圖 + Detailed Label (Overlay) ===
        axes[2].imshow(img, cmap='gray') # 先畫底圖
        if has_detail and lbl_detail is not None:
            # 將背景 (0) 遮罩起來
            masked_detail = ma.masked_where(lbl_detail == 0, lbl_detail)
            # 疊加 Label (使用 tab20 以區分更多類別)
            axes[2].imshow(masked_detail, cmap=cmap_detailed, alpha=0.6, interpolation='nearest')
            axes[2].set_title(f"Overlay: Detailed Label\n(12 Classes)", fontsize=title_fontsize)
        else:
            # 如果這張圖本身就沒有 Detail Label (例如只有 Rough 的資料)
            axes[2].text(0.5, 0.5, "No Detailed Label\n(Source Missing)", ha='center', va='center', color='yellow', fontsize=14, backgroundcolor='black')
            axes[2].set_title("Overlay: Detailed Label", fontsize=title_fontsize)
        axes[2].axis('off')
        
        detailed_patches = [mpatches.Patch(color=POSITION_DETAILED_EMBEDDING_COLORS[i], label=f"{i}: {DETAIL_MUSCLE_NAMES[i]}") 
                   for i in DETAIL_MUSCLE_NAMES.keys() if i > 0] # 不顯示背景
        rough_patches = [mpatches.Patch(color=POSITION_ROUGH_EMBEDDING_COLORS[i], label=f"{i}: {ROUGH_MUSCLE_NAMES[i]}") 
                   for i in ROUGH_MUSCLE_NAMES.keys() if i > 0] #
        
        # 將圖例放在最右邊
        fig.legend(handles=detailed_patches + rough_patches, loc='center right', bbox_to_anchor=(1.1, 0.5), title="Muscle Classes")
    
        
        plt.tight_layout()
        plt.show()